In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("udf").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/17 16:20:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'),
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'),
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'),
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'),
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),
]

schema = ["name", "datetime", "price", "currency"]
df = spark.createDataFrame(data=transactions, schema=schema)
df.createOrReplaceTempView("transactions")

In [3]:
spark.sql("SELECT * FROM transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [9]:
from pyspark.sql.types import LongType

# 방법1
def squared1(n):
    return n * n

spark.udf.register("squared1", squared1, LongType())

# 방법2
from pyspark.sql.functions import udf
@udf('long')
def squared2(n):
    return n * n

spark.udf.register('squared2', squared2)

22/02/17 16:27:43 WARN SimpleFunctionRegistry: The function squared1 replaced a previously registered function.


<function __main__.squared2(n)>

In [10]:
spark.sql("SELECT name, squared1(price), squared2(price) FROM transactions").show()

+--------------------------+---------------+---------------+
|                      name|squared1(price)|squared2(price)|
+--------------------------+---------------+---------------+
|          찹쌀탕수육+짜장2|      484000000|      484000000|
|등심탕수육+크립새우+짜장면|      462250000|      462250000|
|          월남 쌈 2인 세트|     1764000000|     1764000000|
|       콩국수+열무비빔국수|      451562500|      451562500|
|       장어소금+고추장구이|     4719690000|     4719690000|
|                      족발|     1024000000|     1024000000|
+--------------------------+---------------+---------------+



In [13]:
def read_number(n):
    units = ["", "십", "백", "천", "만"]
    nums = "일이삼사오육칠팔구"
    result = []
    i = 0
    while n > 0:
        n, r = divmod(n, 10)
        if r > 0:
            result.append(nums[r-1]+units[i])
        i += 1
    return "".join(result[::-1])

print(read_number(21250))
print(read_number(68700))

이만일천이백오십
육만팔천칠백
